In [398]:
target_url = "https://bsr.sudrf.ru/bigs/portal.html#%7B%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_document_result_date%5C%22,%5C%22operator%5C%22:%5C%22B%5C%22,%5C%22query%5C%22:%5C%222018-01-01T00:00:00%5C%22,%5C%22sQuery%5C%22:%5C%222018-12-31T00:00:00%5C%22%7D,%7B%5C%22name%5C%22:%5C%22case_document_category_article_cat%5C%22,%5C%22operator%5C%22:%5C%22SEW%5C%22,%5C%22query%5C%22:%5C%22%D0%A1%D1%82%D0%B0%D1%82%D1%8C%D1%8F%20229.1%20%D0%A7%D0%B0%D1%81%D1%82%D1%8C%203%5C%22%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22common%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D%5D%7D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22simpleSearchFieldsBundle%22:%22default%22,%22filterGroups%22:%5B%7B%22filterQueries%22:%5B%7B%22field%22:%22case_court_type%22,%22query%22:%22%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9,%20%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D1%81%D0%BA%D0%BE%D0%B9,%20%D0%BC%D0%B5%D0%B6%D1%80%D0%B0%D0%B9%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9%20%D1%81%D1%83%D0%B4%22,%22not%22:false%7D%5D,%22groupMode%22:%22OR%22,%22not%22:false%7D%5D%7D"

In [66]:
logging?

Object `logging` not found.


In [194]:
import logging
import selenium as se
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from utils import LazyProperty
from functools import partial

In [198]:
EC.presence_of_element_located?

Init signature: EC.presence_of_element_located(locator)
Docstring:     
An expectation for checking that an element is present on the DOM
of a page. This does not necessarily mean that the element is visible.
locator - used to find the element
returns the WebElement once it is located
File:           ~/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/selenium/webdriver/support/expected_conditions.py
Type:           type


In [204]:
%%time
url = target_url
options = se.webdriver.ChromeOptions()
options.add_argument('headless')
driver = se.webdriver.Chrome(chrome_options=options)
driver.get(url)
driver.implicitly_wait(30)
elements = driver.find_elements(by=By.CSS_SELECTOR, value=".resultHeader.openCardLink")
elements

/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 10.2 s


In [420]:
logger = logging.getLogger()

class Scraper:
    patterns = {
        "bsr.sudrf.ru": "Уголовное дело"
    }

    element_css_tags = {
        "case_link" : ".resultHeader.openCardLink",
        "file_url": ".bigField"
    }

    def __init__(self, url, pattern_key: str="bsr.sudrf.ru"):
        self.url = url
        self.options = se.webdriver.ChromeOptions()
        self.options.add_argument('headless')
        self.driver = se.webdriver.Chrome(options=self.options)
        self.driver.implicitly_wait(30)
        try:
            self.pattern = self.patterns[pattern_key]
        except BaseException as e:
            logger.warning(e)
            
        self.driver.get(self.url)
        
    def get_elements_by_css_selector(self, element_name: str):
        css_tag = self.element_css_tags.get(element_name)
        elements = self.driver.find_elements(by=By.CSS_SELECTOR, value=css_tag)
        return list(elements)

    @LazyProperty
    def case_links(self) -> list:
        return self.get_elements_by_css_selector("case_link")
    
    def get_url(self, element: se.webdriver.remote.webelement.WebElement) -> str:
        """ Parse current web page"""
        return element.get_attribute("href")
    
    @LazyProperty
    def current_page_case_urls(self) -> list:
        return list(map(self.get_url, self.case_links))
    
    @LazyProperty
    def file_url(self):
        return list(map(self.get_url, self.get_elements_by_css_selector("file_url")))[0]
        
    def screenshot(self, element: se.webdriver.remote.webelement.WebElement):
        """ Saves element screenshot to current directory """
        fname = element.text+"_"+".png"
        element.screenshot(fname)
    
    def go_to_next_page(self):
        pass


search_page = Scraper(target_url)
case_page_url = search_page.current_page_case_urls[0]
case_page = Scraper(case_page_url)
file_page = Scraper(case_page.file_url)

In [421]:
text_elems = file_page.driver.find_elements(by=By.CSS_SELECTOR, value=".width100.height100.field-iframe")[0]
text_elems.tag_name

'iframe'

In [467]:
test_tags = [
    "/html/body/span",
    "/html/body/span/p",
    "/html/body/span/text",
    "/html/body/span/child::li",
    "/html/body/span//child::li",
    "/html/body/span/descendant::li",
    "/html/body/span//descendant::li",
    "/html/body/span/descendant::p",
    "/html/body/span/text",
    "/html/body/span/p[0]",
    "/html/body/span//p",
    "//span[p]",
    "/html/body//span"
]

In [469]:
for tag in test_tags:
    print("{}: {}".format(tag, text_elems.find_elements(by=By.XPATH, value=tag)))

/html/body/span: []
/html/body/span/p: []
/html/body/span/text: []
/html/body/span/child::li: []
/html/body/span//child::li: []
/html/body/span/descendant::li: []
/html/body/span//descendant::li: []
/html/body/span/descendant::p: []
/html/body/span/text: []
/html/body/span/p[0]: []
/html/body/span//p: []
//span[p]: []


In [430]:
value = "/html/body/span"
file_page.driver.find_elements(by=By.XPATH, value=value)

[]

In [423]:
value = "/html/body/span/p"
file_page.driver.find_elements(by=By.XPATH, value=value)

[]

In [422]:
value = "/html/body/span/text"
file_page.driver.find_elements(by=By.XPATH, value=value)

[]

In [426]:
value = "/html/body/span/child::li"
file_page.driver.find_elements(by=By.XPATH, value=value)

[]

In [427]:
value = "/html/body/span//child::li"
file_page.driver.find_elements(by=By.XPATH, value=value)

[]

In [428]:
value = "/html/body/span/descendant::li"
file_page.driver.find_elements(by=By.XPATH, value=value)

[]

In [432]:
value = "/html/body/span//descendant::li"
file_page.driver.find_elements(by=By.XPATH, value=value)

[]

In [431]:
value = "/html/body/span/descendant::p"
file_page.driver.find_elements(by=By.XPATH, value=value)

[]

In [419]:
value = "/html/body/span/text"
file_page.driver.find_elements(by=By.XPATH, value=value)

[]

In [414]:
value = "/html/body/span/p[0]"
text_elems.find_elements(by=By.XPATH, value=value)

[]

In [415]:
value = "/html/body/span//p"
text_elems.find_elements(by=By.XPATH, value=value)

[]

In [443]:
value = "//span[p]" # "/html/body//span"
spans = text_elems.find_elements(by=By.XPATH, value=value)
spans

[]

In [441]:
[s.get_attribute('textContent') for s in spans]

['']

In [447]:
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

response = HtmlResponse(url=case_page_url)
Selector(response=response).xpath('//span/text()').get()

In [455]:
response.xpath('/html/body//span')

[]

In [ ]:
value = "//span//p" # "/html/body//span"
spans = text_elems.find_elements(by=By.XPATH, value=value)
spans

In [393]:
spans[0].find_elements(by=By.TAG_NAME, value="p")

[]

In [456]:
value = "/html/body//span" # "/html/body//span"
spans = text_elems.find_elements(by=By.XPATH, value=value)
spans

[<selenium.webdriver.remote.webelement.WebElement (session="92d9d601b8ab8e239e36fbe25fe6bfbd", element="0.4425051423764661-2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="92d9d601b8ab8e239e36fbe25fe6bfbd", element="0.4425051423764661-3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="92d9d601b8ab8e239e36fbe25fe6bfbd", element="0.4425051423764661-4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="92d9d601b8ab8e239e36fbe25fe6bfbd", element="0.4425051423764661-5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="92d9d601b8ab8e239e36fbe25fe6bfbd", element="0.4425051423764661-6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="92d9d601b8ab8e239e36fbe25fe6bfbd", element="0.4425051423764661-7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="92d9d601b8ab8e239e36fbe25fe6bfbd", element="0.4425051423764661-8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="92d9d601b8ab8e239e36fbe25fe6bfbd", el

In [464]:
text_elems.screenshot("text_elems.png")

True

In [458]:
for span in spans:
    print(span.text)
    print(span.tag_name)

Найти в деле
span
Найти в деле
span
Найти в деле
span

span

span

span

span

span

span

span

span

span

span

span

span

span

span

span

span

span

span

span


In [367]:
texts = spans[0]
texts

IndexError: list index out of range

In [344]:
text = texts[0]

In [411]:
text.text

''

In [ ]:
value = "//*"
texts = texts.find_elements(by=By.XPATH, value=value)
texts

In [380]:
text = texts[0]

In [387]:
text.tag_name

'html'

In [390]:
text.get_attribute("innerhtml")

In [386]:
[t.text for t in texts]

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [351]:
text.tag_name

'span'

In [345]:
text.screenshot("text.png")

True

In [337]:
texts

[]

In [335]:
[t.text for t in texts]

['Найти в деле',
 'Найти в деле',
 'Найти в деле',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [313]:
text_elems.find_elements(by=By.TAG_NAME, value="p")

[]

In [312]:
text_elems.find_element_by_tag_name("p")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"p"}
  (Session info: headless chrome=74.0.3729.169)
  (Driver info: chromedriver=74.0.3729.169 (78e4f8db3ce38f6c26cf56eed7ae9b331fc67ada-refs/branch-heads/3729@{#1013}),platform=Linux 4.10.0-35-generic x86_64)


In [284]:
text_span_elem = text_elem.find_element_by_xpath("//span")

In [285]:
text_span_elem.text

'Найти в деле'

In [79]:
element.screenshot("element.png")

True